# gameplay

> Fill in a module description here

In [1]:
#| default_exp gameplay

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
%load_ext autoreload


In [3]:
%autoreload 2

In [5]:
#| export
import clue.model as model
import clue.ui as ui
from random import choice,shuffle,choices
import os

In [6]:
#| hide
stgame = model.createClue(4)
stmoves = []

In [7]:
#| export
def getPlay(pc,game, moves):
    
    

    ui.showBoard(pc,game,moves)
    playerRoom = model.roomOf(pc,game)
    passages = model.roomsAround(playerRoom)
    locs = [[3,x] for x in passages ]
    verb = choice(["totter","walk","stumble","staggar","rush","dash","race","sprint"])


    print(model.people[pc-1][2] + " will " + verb + " to:\n")
 
    l = ui.chooseKey(locs,pc,game,moves)
    l = passages[l-1]

    p, w = ui.choosePandW(pc,game,moves)
    


    return p-1, w-1, l



In [8]:
pgame = model.createClue(5)
print(ui.showChoice(getPlay(1,pgame,[])))

╒════════════╤═══════════════╤══════════════╕
│ Ball Room  │ Billiard Room │ Conservatory │
│    G       │               │    P         │
├────────────┼───────────────┼──────────────┤
│ 3) Study   │               │ Dining Room  │
├────────────┼───────────────┼──────────────┤
│ 2) Library │               │ Hall         │
│    S       │               │    W         │
├────────────┼───────────────┼──────────────┤
│            │ 1) Lounge     │ Kitchen      │
│            │    M          │    P         │
╘════════════╧═══════════════╧══════════════╛
👠 will race to:

1 Lounge
2 Library
3*Study


Choose from 1 to 3:  3


    People              Weapons
--  ------------------  -------------
 1  👠 Miss Scarlet     🔪 Knife
 2  🧐 Colonel Mustard  *🔫 Revolver
 3  🥼  Mrs. White      🪢 Rope
 4  *🐉 Mr. Green       *🔧 Wrench
 5  🍆 Professor Plum   🕯️ Candlestick
 6  🦚 Mrs. Peacock     🪠 Lead Pipe


 3 5


🥼  Mrs. White with the 🕯️ Candlestick in the Study


In [9]:
#| hide
#stguess = getPlay(1,stgame,stmoves)
#print(ui.showChoice(stguess))
def seenCounts(pc,game, moves):
    knoweledge = ui.getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

In [10]:
#| hide
#stguess = getPlay(2,stgame,stmoves)

In [10]:
#| export




def simpleGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [11]:
#| export

def itemsForType(t):
    if t == 1:
        return model.people
    if t == 2:
        return model.weapons
    return model.rooms
        
def itemGuess(t,moves):
    l = itemsForType(t)

    ret = [1 for x in range(len(l))]
    for x in moves:
        ret[x[t-1]] += 1
    #print(ret)
    return ret
    
def showCounts(t,moves):
    items = itemGuess(t,moves)
    base = itemsForType(t)
    #print(items,base)
    for i in range(len(items)):
        print(i,ui.cardFull(base[i]),items[i])


                              




In [12]:
#| export
def biasWalk(a,b,c):
    
    def bWalk(exclude,pc,game, moves):
        playerRoom = model.roomOf(pc,game) 
        dest = model.roomsAround(playerRoom)
        if model.keyCard(3,dest[a]) not in exclude:
            return dest[a]%100
        if model.keyCard(3,dest[b]) not in exclude:
            return dest[b]%100
        
        return dest[c]%100
    return bWalk
    
    
 

In [13]:
#| export
def rWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
                              
    return l

In [14]:
#| export


def roomsAway(roomNum,roomSize):
    foo = [x for x in range(roomSize)]
    middle = int( roomSize / 2)
    if roomNum <middle:
        lm = foo[roomNum - int(middle):]
        lSide = lm + foo[:roomNum] 
        lSide.reverse()
        rSide = foo[roomNum + 1:roomNum + 1 + middle]
    elif roomNum > middle:
        lSide = foo[roomNum-middle:roomNum]
        lSide.reverse()
        rSide = foo[roomNum +1: ] + foo[:roomNum-middle]
    else:
        lSide = foo[:middle]
        lSide.reverse()
        rSide = foo[middle+1:]
    return lSide, rSide

def repelWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    lRooms, rRooms = roomsAway(playerRoom,len(model.rooms))
    #print("staring at " + ui.cardFull(model.rooms[playerRoom]))
    #print(playerRoom,lRooms,rRooms)
    if not (model.keyCard(3,playerRoom) in exclude):
     return playerRoom
    #print(exclude)
    def score(rooms):
        rooms.reverse()
        weights = []
        cards = []
        for i in range(len(rooms)):
            r = rooms[i]
            cards.append(str(model.keyCard(3,r)) + " " + model.rooms[r][0])
            
            if model.keyCard(3,r) in exclude:
                w = 2**i
                #print(i,w)
                weights.append(w)
            else:
                weights.append(0)
        #print(cards,weights)
        return sum(weights)


    lPull = score(lRooms)
    rPull = score(rRooms)
    #trace.append([playerRoom,lPull,rPull])
    #print(trace)
    #ui.showBoard(pc,game,moves)
    achoice = model.roomsAround(playerRoom)
    if lPull > rPull:
        #print("to" + ui.cardFull(model.rooms[achoice[2]]))
        return achoice[2]
    if lPull < rPull:
        
        #print("to" + ui.cardFull(model.rooms[achoice[0]]))
        return achoice[0]
    #print("same")    
    return rWalk(exclude,pc,game, moves)


In [15]:
#| export
def mimic(memory,walk):
    def betterGuess(pc,game, moves):

        exclude  = set(model.getSeen(pc,game, moves))

        if memory < 0:
            examine = []
        else:
            examine = moves[-memory:]


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    
    return betterGuess

In [16]:
#| export
def cheater(walk):
    def betterGuess(pc,game, moves):

 
        exclude  = set(model.handOf(pc,game) +[x[-1] for x in moves] )

       
        examine = []
   


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    return betterGuess


def halfRight(walk):
    oracle = cheater(walk)
    
    def betterGuess(pc,game, moves):
        p,w,l = oracle(pc,game, moves)
        exclude  = model.handOf(pc,game) +[x[-1] for x in moves] 
        exclude = [x for x in exclude if x < 300]
        

        if len(exclude) <= 0:
            return (p,w,l)

        wrong = choice(exclude)
        if wrong < 200:
            p = wrong%100
        else:
            w = wrong%100

        return (p,w,l)
    return betterGuess

In [17]:
#| export

def hasGuess(guess,moves):
    p, w, l = guess[:3]
    c = (p,w,l)
    for x in moves:
        p, w, l = x[:3]
        xc = (p,w,l)
        if c == xc:
            return True
    return False

def repeater(strats,verbose=False):
    def betterGuess(pc,game, moves):
        counter = 0
        for strat in strats:
            
            g =  strat(pc,game, moves)
            if not hasGuess(g,moves):
                if verbose and counter > 0:
                    print("counter " + str(counter))
                return g
            counter += 1
                        
        if verbose and counter > 0:
            print(" over counter " + str(counter))
        return strats[-1](pc,game, moves)
                          
    return betterGuess

In [18]:
r1 = repeater([mimic(1*(x+3),rWalk) for x in range(4)])
rgame = model.createClue(4)
r1(1,rgame,[])

(5, 4, 0)

## Visual

In [19]:
#| export
def showHist(pc,game, moves):
    amoves = [x for x in moves if x[-3] == pc]
    ui.showMoves(pc,amoves,len(game)-1)

In [20]:
#| export
def playerName(i):
    return ui.cardFull(model.cardLookup(1,i-1))
    
def runVisualGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1

   
             
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        if isUser:
            #ui.showMoves(player,amoves[-numPlayers:],numPlayers)
            print("your moves:")
            showHist(player,agame, amoves)
            print("your hand:")
            ui.printPlayerHand(player,agame)
            ui.showMoves(player,amoves[-numPlayers+2:],numPlayers)

        #print(player,agame,amoves)
        ans = strat(player,agame,amoves)
        if isUser:
            os.system("clear")
            #print(seenCounts(player,agame,amoves))
            print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        #print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    print("\n\n" + playerName(player) + " is not quite ready to accuse.")
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)


        if not done:
            player = player + 1
            if player - 1 >=len(strats):
               player = 1 
    
    
    ui.printGame(agame) 
    ui.showMoves(-1,amoves,numPlayers)
    print("\n" + playerName(player) + " " + stratName + " won!")
    return agame, amoves

In [24]:
def quickGame():
    p1 = ("user",True,getPlay)
    s1 = ("simple",False,mimic(4,rWalk))
    hr1 = ("half",False,halfRight(rWalk))
    s2 = ("simple r",False,repeater([mimic(-1,rWalk) for x in range(4)]))
    b2 = ("better rZ",False,repeater([mimic(8,rWalk) for x in range(4)]))

    strategies = [b2,hr1,p1,s1,s2]
    runVisualGame(strategies)

In [ ]:
quickGame()

your moves:
your hand:
🧐 Colonel Mustard, 🔫 Revolver, Conservatory, 🪢 Rope
👠  asks (🦚 &  🔪 in the Ball Room) ----->👠🧐❓❓❓
🧐  asks (🦚 &  🕯️ in the Billiard Room) ->❓🧐🥼🐉❓
╒════════════╤═══════════════╤══════════════╕
│ Ball Room  │ Billiard Room │ Conservatory │
│    S       │    M, P       │              │
├────────────┼───────────────┼──────────────┤
│ 3) Study   │               │ Dining Room  │
│    G       │               │    P         │
├────────────┼───────────────┼──────────────┤
│ 2) Library │               │ Hall         │
│    W       │               │              │
├────────────┼───────────────┼──────────────┤
│            │ 1) Lounge     │ Kitchen      │
╘════════════╧═══════════════╧══════════════╛
🥼 will staggar to:

1 Lounge
2 Library
3 Study


Choose from 1 to 3:  2


    People              Weapons
--  ------------------  -------------
 1  👠 Miss Scarlet     🔪 Knife
 2  🧐 Colonel Mustard  🔫 Revolver
 3  🥼  Mrs. White      🪢 Rope
 4  🐉 Mr. Green        *🔧 Wrench
 5  *🍆 Professor Plum  🕯️ Candlestick
 6  *🦚 Mrs. Peacock    🪠 Lead Pipe


 2 2


🥼  Mrs. White suspects 🧐 Colonel Mustard with the 🔫 Revolver in the Library.
your moves:
🥼  asks (🧐 &  🔫 in the Library) ------->👠❓🥼🐉🍆🦚 shows Library
your hand:
🧐 Colonel Mustard, 🔫 Revolver, Conservatory, 🪢 Rope
🍆  asks (🐉 &  🪢 in the Hall) ---------->👠❓❓❓🍆
👠  asks (👠 &  🕯️ in the Study) --------->👠🧐🥼🐉❓
🧐  asks (🍆 &  🔫 in the Lounge) -------->❓🧐🥼❓❓
╒════════════╤═══════════════╤══════════════╕
│ Ball Room  │ Billiard Room │ Conservatory │
│            │    P          │              │
├────────────┼───────────────┼──────────────┤
│ 3) Study   │               │ Dining Room  │
│    S       │               │              │
├────────────┼───────────────┼──────────────┤
│ 2) Library │               │ Hall         │
│    W       │               │    G         │
├────────────┼───────────────┼──────────────┤
│            │ 1) Lounge     │ Kitchen      │
│            │    M, P       │              │
╘════════════╧═══════════════╧══════════════╛
🥼 will staggar to:

1 Lounge
2*Library            

...

    People              Weapons
--  ------------------  -------------
 1  👠 Miss Scarlet     🔪 Knife
 2  🧐 Colonel Mustard  *🔫 Revolver
 3  🥼  Mrs. White      🪢 Rope
 4  🐉 Mr. Green        *🔧 Wrench
 5  *🍆 Professor Plum  🕯️ Candlestick
 6  *🦚 Mrs. Peacock    🪠 Lead Pipe


 1 1


🥼  Mrs. White suspects 👠 Miss Scarlet with the 🔪 Knife in the Lounge.
your moves:
🥼  asks (🧐 &  🔫 in the Library) ------->👠❓🥼🐉🍆🦚 shows Library
🥼  asks (👠 &  🔪 in the Lounge) -------->❓❓🥼🐉❓❓ shows 👠 Miss Scarlet
your hand:
🧐 Colonel Mustard, 🔫 Revolver, Conservatory, 🪢 Rope
🍆  asks (🦚 &  🪢 in the Library) ------->👠❓❓❓🍆
👠  asks (🧐 &  🕯️ in the Kitchen) ------->👠🧐🥼❓❓
🧐  asks (🥼 &  🔫 in the Lounge) -------->❓🧐🥼❓❓
╒════════════╤═══════════════╤══════════════╕
│ Ball Room  │ Billiard Room │ Conservatory │
├────────────┼───────────────┼──────────────┤
│ Study      │               │ Dining Room  │
│            │               │    G         │
├────────────┼───────────────┼──────────────┤
│ 3) Library │               │ Hall         │
│    P, P    │               │              │
├────────────┼───────────────┼──────────────┤
│            │ 2) Lounge     │ 1) Kitchen   │
│            │    M, W       │    S         │
╘════════════╧═══════════════╧══════════════╛
🥼 will rush to:

1 Kitchen
2 Lounge

Choose from 1 to 3:  2


    People                                     Weapons
--  -----------------------------------------  -------------
 1  👠 Miss Scarlet                            🔪 Knife
 2  🧐 Colonel Mustard                 👠      *🔫 Revolver
 3  🥼  Mrs. White                             🪢 Rope
 4  🐉 Mr. Green                       🐉, 🍆  *🔧 Wrench
 5  *🍆 Professor Plum                         🕯️ Candlestick
 6  *🦚 Mrs. Peacock                           🪠 Lead Pipe


In [23]:
p1 = ("player",True,getPlay)
s1 = ("simple",False,mimic(-1,rWalk))
b1 = ("better",False,mimic(2,rWalk))
d1 =("deep",False,mimic(8,rWalk))
c1 = ("cheater",False,cheater(repelWalk))
s2 = ("simpleR",False,mimic(4,rWalk))
hr = ("halfRight",False,halfRight(rWalk))
s3 = ("stay",False,mimic(-1,biasWalk(1,0,2)))
sleft =  ("sleft",False,mimic(-1,biasWalk(1,0,0)))
strategies = [s1, s2,b1,b1,d1,s3]
strategies = [s3, s2,sleft,s3]

In [44]:
#| hide

strategies = [ s1,p1,s1,hr,hr] #hr,s1,p1,s1,s1]
rgame, rmoves = runVisualGame(strategies)

your moves:
your hand:
🍆 Professor Plum, Ball Room, Study, 👠 Miss Scarlet
👠  asks (🦚 &  🪢 in the Dining Room) --->👠🧐🥼🐉❓
╒═══════════╤═══════════════╤═════════════════╕
│ Ball Room │ Billiard Room │ 1) Conservatory │
│    G      │    W          │                 │
├───────────┼───────────────┼─────────────────┤
│ Study     │               │ 2) Dining Room  │
│           │               │    S, M, P      │
├───────────┼───────────────┼─────────────────┤
│ Library   │               │ 3) Hall         │
│    P      │               │                 │
├───────────┼───────────────┼─────────────────┤
│           │ Lounge        │ Kitchen         │
╘═══════════╧═══════════════╧═════════════════╛
🧐 will stumble to:

1 Conservatory
2 Dining Room                      👠, 🧐
3 Hall


Choose from 1 to 3:  2


    People              Weapons
--  ------------------  -------------
 1  *👠 Miss Scarlet    🔪 Knife
 2  🧐 Colonel Mustard  🔫 Revolver
 3  🥼  Mrs. White      🪢 Rope
 4  🐉 Mr. Green        🔧 Wrench
 5  *🍆 Professor Plum  🕯️ Candlestick
 6  🦚 Mrs. Peacock     🪠 Lead Pipe


 2 3


🧐 Colonel Mustard suspects 🧐 Colonel Mustard with the 🪢 Rope in the Dining Room.
your moves:
🧐  asks (🧐 &  🪢 in the Dining Room) --->❓🧐🥼❓❓❓ shows 🧐 Colonel Mustard
your hand:
🍆 Professor Plum, Ball Room, Study, 👠 Miss Scarlet
🧐  asks (🧐 &  🪢 in the Dining Room) --->❓🧐🥼❓❓ shows 🧐 Colonel Mustard
🥼  asks (🐉 &  🪠 in the Conservatory) -->👠❓🥼🐉🍆
🐉  asks (🧐 &  🔫 in the Billiard Room) ->👠❓❓🐉🍆
🍆  asks (👠 &  🔧 in the Lounge) -------->👠🧐❓❓🍆
👠  asks (🥼 &  🪠 in the Library) ------->👠🧐🥼❓❓
╒══════════════╤══════════════════╤═════════════════╕
│ 1) Ball Room │ 2) Billiard Room │ 3) Conservatory │
│              │    M, G          │                 │
├──────────────┼──────────────────┼─────────────────┤
│ Study        │                  │ Dining Room     │
│              │                  │    P            │
├──────────────┼──────────────────┼─────────────────┤
│ Library      │                  │ Hall            │
│    S, W      │                  │                 │
├──────────────┼──────────────────

Choose from 1 to 3:  2


    People                                  Weapons
--  --------------------------------------  -------------
 1  *👠 Miss Scarlet                    🍆  🔪 Knife
 2  *🧐 Colonel Mustard                     🔫 Revolver
 3  🥼  Mrs. White                          🪢 Rope
 4  🐉 Mr. Green                            🔧 Wrench
 5  *🍆 Professor Plum                      🕯️ Candlestick
 6  🦚 Mrs. Peacock                         🪠 Lead Pipe


 3 3


🧐 Colonel Mustard suspects 🥼  Mrs. White with the 🪢 Rope in the Billiard Room.
your moves:
🧐  asks (🧐 &  🪢 in the Dining Room) --->❓🧐🥼❓❓❓ shows 🧐 Colonel Mustard
🧐  asks (🥼 &  🪢 in the Billiard Room) ->❓🧐🥼🐉❓❓ shows 🪢 Rope
your hand:
🍆 Professor Plum, Ball Room, Study, 👠 Miss Scarlet
🧐  asks (🥼 &  🪢 in the Billiard Room) ->❓🧐🥼🐉❓ shows 🪢 Rope
🥼  asks (👠 &  🔪 in the Billiard Room) ->❓❓🥼🐉❓
🐉  asks (🐉 &  🪢 in the Ball Room) ----->👠❓❓🐉🍆
🍆  asks (🧐 &  🪠 in the Lounge) -------->👠🧐🥼❓🍆
👠  asks (🥼 &  🔪 in the Ball Room) ----->👠🧐❓❓❓
╒════════════╤═══════════════╤══════════════╕
│ Ball Room  │ Billiard Room │ Conservatory │
│    S, W, G │               │              │
├────────────┼───────────────┼──────────────┤
│ Study      │               │ Dining Room  │
│            │               │    P         │
├────────────┼───────────────┼──────────────┤
│ 3) Library │               │ Hall         │
├────────────┼───────────────┼──────────────┤
│            │ 2) Lounge     │ 1) Kitchen   │
│            

KeyboardInterrupt: Interrupted by user

In [ ]:
 ui.showMoves(-1,rmoves,4)

👠  asks (🥼 &  🔧 in the Dining Room) --->👠🧐🥼🐉 shows Dining Room
🧐  asks (🥼 &  🔧 in the Dining Room) --->❓🧐🥼❓ shows Kitchen
🥼  asks (🥼 &  🔧 in the Dining Room) --->❓❓🥼🐉 shows 🔧 Wrench
🐉  asks (🦚 &  🪢 in the Conservatory) -->👠🧐🥼🐉 shows Lounge
🍆  asks (🦚 &  🔧 in the Dining Room) --->👠🧐🥼❓ shows Hall
👠  asks (🍆 &  🔧 in the Dining Room) --->👠🧐🥼🐉 shows 🔧 Wrench
🧐  asks (🧐 &  🪢 in the Conservatory) -->❓🧐🥼❓ shows Hall
🥼  asks (🧐 &  🪢 in the Conservatory) -->❓❓🥼🐉 shows Dining Room
🐉  asks (🐉 &  🔧 in the Dining Room) --->👠❓❓🐉 shows Library
🍆  asks (🐉 &  🔪 in the Ball Room) ----->👠🧐❓❓ shows 🐉 Mr. Green
👠  asks (🥼 &  🪢 in the Conservatory) -->👠🧐🥼❓ shows Billiard Room
🧐  asks (🧐 &  🔧 in the Dining Room) --->❓🧐🥼🐉 shows 🔧 Wrench


KeyboardInterrupt: 

In [ ]:
#| export
def keysForGuess(guess):
    p,w,l = guess[:3]
    h = [[1,p],[2,w],[3,l]]
    h = [model.keyCard(x[0],x[1]) for x in h]
    return h

def responseToGuess(guess,asker,answered,shown):
    p,w,l = guess
    return (p,w,l,asker, answered,shown)
def localgetAnswer(playerNum,game,guess):

    h = keysForGuess(guess)
    shuffle(h)
    print(h)

    playCount = len(game) - 1
    
    c = playerNum+1
    
    if c > playCount:
        c = 1
    while c != playerNum:
        hand = game[c]
        print(hand)
        for y in h:
            for x in hand:
                 

                if x == y :
                    return responseToGuess(guess,playerNum,c,y)
        c = c+1
        if c >= playCount:
            c = 1
    return (-1,0,0)

In [ ]:
check = localgetAnswer(2,rgame,(3, 4, 5))
check,  ui.showMoves(-1,[check],4)

[204, 103, 305]
[306, 304, 305, 301]
🧐  asks (🐉 &  🕯️ in the Hall) ---------->❓🧐🥼❓ shows Kitchen


((3, 4, 5, 2, 3, 305), None)

In [ ]:
#| export
def runGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
          
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        ans = strat(player,agame,amoves)
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)

        if not done:
            player = player + 1
            if player - 1 >= numPlayers:
               player = 1 
    
    return player, agame , amoves


In [ ]:
#| hide
s1 = ("simple",False,mimic(-1,rWalk))
b1 = ("better",False,mimic(2,rWalk))
d1 =("deep",False,mimic(8,rWalk))
c1 = ("cheater",False,cheater(repelWalk))
s2 = ("simpleR",False,mimic(4,rWalk))
hr = ("halfRight",False,halfRight(rWalk))
s3 = ("stay",False,mimic(-1,biasWalk(1,0,2)))
sleft =  ("sleft",False,mimic(-1,biasWalk(1,0,0)))
strategies = [s1, s2,b1,b1,d1,s3]
strategies = [s3, s2,sleft,s3]
counts = [0 for x in range(len(strategies))]
for c  in range(1000):
    p, g, m = runGame(strategies)
    counts[p-1] += 1

m = zip([x[0] for x in strategies],counts)
for x in m:
    print(x)

In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()